# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time



# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

Weather_data = pd.read_csv("../Weather Api/Output/weather.csv")
Weather_data

Dropna_weather_data = Weather_data.dropna()
Dropna_weather_data.head(20)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.08,-159.32,80.60,69,90,4.70,US,1604199757
1,Qaanaaq,77.48,-69.36,5.14,81,0,4.81,GL,1604199738
2,Sechura,-5.56,-80.82,64.65,81,55,10.83,PE,1604199757
3,Rikitea,-23.12,-134.97,76.06,68,0,8.55,PF,1604199757
4,Punta Arenas,-53.15,-70.92,48.20,81,0,6.93,CL,1604199758
5,Bluff,-46.60,168.33,53.01,84,99,15.99,NZ,1604199758
6,Birao,10.28,22.79,73.67,59,0,1.54,CF,1604199758
7,Krasnyye Tkachi,57.49,39.76,35.60,91,90,6.71,RU,1604199758
8,Puerto Escondido,15.85,-97.07,79.18,75,4,4.72,MX,1604199758
9,Richards Bay,-28.78,32.04,64.02,87,100,3.47,ZA,1604199496


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = Weather_data[['Lat','Lng']]

humidity = Weather_data['Humidity']

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = Dropna_weather_data.loc[(Dropna_weather_data["Max Temp"] > 70) & (Dropna_weather_data["Max Temp"] < 80) & (Dropna_weather_data["Cloudiness"] == 0), :]
narrowed_city_df = Dropna_weather_data.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.08,-159.32,80.60,69,90,4.70,US,1604199757
1,Qaanaaq,77.48,-69.36,5.14,81,0,4.81,GL,1604199738
2,Sechura,-5.56,-80.82,64.65,81,55,10.83,PE,1604199757
3,Rikitea,-23.12,-134.97,76.06,68,0,8.55,PF,1604199757
4,Punta Arenas,-53.15,-70.92,48.20,81,0,6.93,CL,1604199758
...,...,...,...,...,...,...,...,...,...
573,Bherāmāra,24.02,88.99,82.51,73,10,3.67,BD,1604199847
574,Balikpapan,-1.27,116.83,84.20,74,20,5.82,ID,1604199847
575,Garhwa,24.18,83.82,75.51,56,0,4.90,IN,1604199847
576,Skibbereen,51.55,-9.27,53.60,100,90,28.86,IE,1604199847


In [6]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(narrowed_city_df["City"])):

    lat = narrowed_city_df.loc[city]["Lat"]
    lng = narrowed_city_df.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
narrowed_city_df["Nearest Hotel"] = hotels
narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel
0,Kapaa,22.08,-159.32,80.60,69,90,4.70,US,1604199757,Sheraton Kauai Resort at Coconut Beach
1,Qaanaaq,77.48,-69.36,5.14,81,0,4.81,GL,1604199738,Qaanaaq Hotel
2,Sechura,-5.56,-80.82,64.65,81,55,10.83,PE,1604199757,Hospedaje Costa Bella
3,Rikitea,-23.12,-134.97,76.06,68,0,8.55,PF,1604199757,Pension Maro'i
4,Punta Arenas,-53.15,-70.92,48.20,81,0,6.93,CL,1604199758,Hotel Dreams Del Estrecho
...,...,...,...,...,...,...,...,...,...,...
573,Bherāmāra,24.02,88.99,82.51,73,10,3.67,BD,1604199847,PROTAB HEAR DRESER
574,Balikpapan,-1.27,116.83,84.20,74,20,5.82,ID,1604199847,Grand Jatra Hotel Balikpapan
575,Garhwa,24.18,83.82,75.51,56,0,4.90,IN,1604199847,A.PS.D Utsav Garden
576,Skibbereen,51.55,-9.27,53.60,100,90,28.86,IE,1604199847,The West Cork Hotel


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
hotel_df = narrowed_city_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel,Hotel Name
0,Kapaa,22.08,-159.32,80.60,69,90,4.70,US,1604199757,Sheraton Kauai Resort at Coconut Beach,Kapaʻa
1,Qaanaaq,77.48,-69.36,5.14,81,0,4.81,GL,1604199738,Qaanaaq Hotel,Qaanaaq
2,Sechura,-5.56,-80.82,64.65,81,55,10.83,PE,1604199757,Hospedaje Costa Bella,Sechura District
3,Rikitea,-23.12,-134.97,76.06,68,0,8.55,PF,1604199757,Pension Maro'i,Rikitea
4,Punta Arenas,-53.15,-70.92,48.20,81,0,6.93,CL,1604199758,Hotel Dreams Del Estrecho,Punta Arenas
...,...,...,...,...,...,...,...,...,...,...,...
573,Bherāmāra,24.02,88.99,82.51,73,10,3.67,BD,1604199847,PROTAB HEAR DRESER,Bheramara
574,Balikpapan,-1.27,116.83,84.20,74,20,5.82,ID,1604199847,Grand Jatra Hotel Balikpapan,Balikpapan
575,Garhwa,24.18,83.82,75.51,56,0,4.90,IN,1604199847,A.PS.D Utsav Garden,Garhwa
576,Skibbereen,51.55,-9.27,53.60,100,90,28.86,IE,1604199847,The West Cork Hotel,Skibbereen


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))